In [277]:
Query="What are all the education qualification of shyam?"

In [278]:
from dotenv import load_dotenv

load_dotenv()


True

In [295]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def process_pdf_batch(pdf_file):
    loader = PyPDFLoader(pdf_file)
    pages = loader.load_and_split()
    character_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0
        )
    pages = character_splitter.split_documents(pages)
    return pages

In [296]:
d="resume"

In [297]:
chunks=process_pdf_batch(f"data\{d}.pdf")

<string>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
<string>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\shyams\AppData\Local\Temp\ipykernel_14848\2212597078.py:1: SyntaxWarning: invalid escape sequence '\{'
  chunks=process_pdf_batch(f"data\{d}.pdf")


In [298]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
if not os.path.exists(f"my_index"):
    document_in_faiss = FAISS.from_documents(chunks, OpenAIEmbeddings())
    document_in_faiss.save_local(f"my_index")

In [299]:
document_in_faiss=FAISS.load_local("my_index", OpenAIEmbeddings(),allow_dangerous_deserialization=True)
faiss_retriever=document_in_faiss.as_retriever(search_kwargs={'k': 10})

In [300]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads

def generate_queries():
        
    # Multi Query: Different Perspectives
    template = """You are an AI language model assistant. Your task is to generate Four 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Original question: {question}"""
    prompt_perspectives = ChatPromptTemplate.from_template(template)


    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
    )
    return generate_querie 


def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]


def _get_docs(m):
    docs=[]
    docs.extend(m['original'])
    for i in m['new']:
        docs.extend(i)
    return docs



In [301]:
from langchain_core.runnables import RunnableParallel
original_question= faiss_retriever
retrieval_chain =  generate_queries() | faiss_retriever.map()
map_chain = RunnableParallel(original=original_question,new=retrieval_chain) | _get_docs

In [302]:
t=map_chain.invoke(Query)

In [303]:
len(t)

20

In [304]:
t

[Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'source': 'data\\resum

In [306]:
t[0]

Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'source': 'data\\resume

# KEYWORDS BM25

In [307]:
def keyword_extractor():
    prompt="""
    You are an AI language model assistant. Your task is to help the user retrieve keywords from their query. 

    Please provide me with the keywords you would like to extract from your query. 

    Keywords: {keywords}
    """
    prompt_perspectives=ChatPromptTemplate.from_template(prompt)
    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() )
    return generate_querie

In [308]:
from langchain_community.retrievers import BM25Retriever
Bm25_retriever = BM25Retriever.from_documents(chunks)
Bm25_retriever.k = 10

In [309]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[Bm25_retriever, map_chain], weights=[0.5, 0.5]
)

In [310]:
final=ensemble_retriever.invoke(Query)

In [311]:
final

[Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'source': 'data\\resum

In [312]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [313]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

compressed_docs = compression_retriever.invoke(Query)
pretty_print_docs(compressed_docs)

c:\Users\shyams\Downloads\RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Document 1:

Shyam Sundar
5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009
♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio
Projects
LLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023
•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.
•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.
•Incorporated FAISS for refined recommendation processes.
PeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023
•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.
•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.
•Achieved 80% prediction accuracy using LightGBM.
--------------------------------------------------------

In [321]:
final_prompt="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

final_prompt_perspectives=ChatPromptTemplate.from_template(final_prompt)

In [322]:
llm_chain = final_prompt_perspectives | ChatOpenAI(temperature=0) | StrOutputParser()

In [324]:
llm_chain.invoke({"question":Query, "context":compressed_docs})

'Shyam has a B.Tech in Chemical Engineering from the National Institute of Technology Tiruchirappalli with a GPA of 7.65 in May 2021 and an M.Tech in Modelling and Simulation from the Defence Institute of Advanced Technology Pune with a GPA of 7.95 in May 2023.'